<a href="https://colab.research.google.com/github/georgievw/ML/blob/main/Degree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#Начало
!pip install Natasha
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 90.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=a7e40490e01002e66733c09d013c67a9ff7327fd1c116d7d0114c34d77f96acd
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26114 sha256=94dd08879976df3e2011ebdd23805c6e4059d1ba88c7db538e3e637cf2a60934
  Stored in directory: /root/.cache/pip/wheels/ab/fa/1b/75d9a71327979678

In [9]:
import os
from PyPDF2 import PdfReader
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    #NewsNERTagger,
    #PER,
    #NamesExtractor,
    Doc
)
#Токенизатор?
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
#ner_tagger = NewsNERTagger(emb)
#names_extractor = NamesExtractor(morph_vocab)

In [10]:
class Span:
  def __init__(self, tokens=list()):
    self.tokens = tokens

  def add_position_token(self, new_token, old_token = None, place = None):
    if len(self.tokens) == 0:
      self.tokens.append(new_token)
      self.root = new_token
    else:
      self.tokens.insert(self.tokens.index(old_token)+place, new_token)
    return

  def add_token(self, token):
    if len(self.tokens) == 0:
      self.root = token
    self.tokens.append(token)
    return

  def pop_first_token(self):
    return self.tokens.pop(0)
  
  @property
  def lemma(self):
    for token in self.tokens:
      token.lemmatize(morph_vocab)
      token.text = token.lemma
      if token is self.root:
        break
    return self.text #TODO
  
  @property
  def text(self):
    return ' '.join(token.text for token in self.tokens)

In [11]:
class Fact:
  def __init__(self):
    self.obj = Span([])
    self.rel = Span([])
    self.subj = Span([])
  
  def is_complete(self):
    if len(self.obj.text) > 0 and len(self.subj.text) > 0 and len(self.rel.text) > 0:
      return True
    else:
      return False

  def to_text(self):
    if self.obj.tokens[0].rel == "case":
      self.rel.add_token(self.obj.pop_first_token())
    return self.subj.lemma.lower() + ',' + self.rel.text.lower() + ',' + self.obj.lemma.lower()

  def to_tuple(self):
    return (self.subj.lemma.lower(), self.rel.text.lower(), self.obj.lemma.lower())

In [12]:
class Sentence:
  def __init__(self, sent):
    self.tokens = [0]
    self.tokens.extend(sent.tokens)
    self.syntax_graph = list()
    self.construct_syntax_graph(sent)
    self.fact = Fact()
    #print(sent.text) #test
    #print(self.syntax_graph) PRINT

  def construct_syntax_graph(self, sent):
    self.syntax_graph = [list() for i in range(len(sent.tokens)+1)]
    for token in sent.tokens:
      self.syntax_graph[int(token.head_id.split('_')[1])].append(int(token.id.split('_')[1]))
    return self.syntax_graph

  def test_root(self):
    #print('1:', self.fact.to_text(), id(self.fact))
    #self.fact = 0
    self.fact = Fact()
    #print('2:', self.fact.to_text(), id(self.fact))
    if self.syntax_graph[0]:
      root_index = self.syntax_graph[0][0]
      self.find_relation(self.fact, root_index)
      #print(self.tokens[fact[1]].text, self.tokens[fact[1]].rel) PRINT
    else:
      print("ERROR: No root token")
      return
    #print(self.tokens[root_index].text, self.tokens[root_index].rel)
    for next_token in self.syntax_graph[root_index]:
      if self.tokens[next_token].rel[:5] == "nsubj":
        self.find_subject(self.fact, next_token)
      elif self.tokens[next_token].rel in  ("obj", "obl", 'xcomp'):
        self.find_object(self.fact, next_token)
      #print(self.tokens[next_token].text, self.tokens[next_token].rel) #PRINT

    if self.fact.is_complete():
      print("OK:", self.fact.to_text())
      print(self.fact)
    else:
      #print("ERROR:", self.fact.to_text())
      return None

    return self.fact

  def find_relation(self, fact, index):
    fact.rel.add_token(self.tokens[index])
    for next_token in self.syntax_graph[index]:
      if self.tokens[next_token].rel == 'fixed':#'xcomp':
        fact.rel.add_token(self.tokens[next_token])

  def find_subject(self, fact, index):
    fact.subj.add_token(self.tokens[index])
    for next_token in self.syntax_graph[index]:
      if self.tokens[next_token].rel in ('amod', 'case'):
        fact.subj.add_position_token(self.tokens[next_token], self.tokens[index], 0)
      elif self.tokens[next_token].rel == 'nmod':
        self.find_subject(fact, next_token)
    return 0

  def find_object(self, fact, index):
    fact.obj.add_token(self.tokens[index])
    for next_token in self.syntax_graph[index]:
      if self.tokens[next_token].rel in ('amod', 'case'):
        fact.obj.add_position_token(self.tokens[next_token], self.tokens[index], 0)
      elif self.tokens[next_token].rel == 'nmod':
        self.find_object(fact, next_token)
    return 0

  def print_paths(self, path="", cur_node=0):
    if not self.syntax_graph[cur_node]:
      print(path+"#")
      return
    for next_node in self.syntax_graph[cur_node]:
      self.print_paths(path+self.tokens[next_node].text+'('+self.tokens[next_node].rel+')', next_node) 


In [13]:
class FactExtractor:
  def __init__(self):
    self.facts = list()
    self.context = list()

  def extract(self, text=""):
    for dirpath, dirnames, filenames in os.walk("./Folder1"):
    # перебрать каталоги
    #for dirname in dirnames:
    #    print("Каталог:", os.path.join(dirpath, dirname))
    # перебрать файлы
      for filename in filenames:
          print("Файл:", os.path.join(dirpath, filename))
          if filename.split('.')[1]== "txt":
            with open(os.path.join(dirpath, filename), 'r') as f:
              text = f.read()
              doc = Doc(text)
              doc.segment(segmenter)
              doc.tag_morph(morph_tagger)
              doc.parse_syntax(syntax_parser)
             #sent = doc.sents[0]
              for sent in doc.sents:
                s = Sentence(sent)
               #print(s.__dict__)
                f = s.test_root()
                if f is not None:
                  self.facts.append(f)
          elif filename.split('.')[1]== "pdf":
            reader = PdfReader(os.path.join(dirpath, filename))
            number_of_pages = len(reader.pages)
            for page in reader.pages:
              text = page.extract_text()
              doc = Doc(text)
              doc.segment(segmenter)
              doc.tag_morph(morph_tagger)
              doc.parse_syntax(syntax_parser)
             #sent = doc.sents[0]
              for sent in doc.sents:
                s = Sentence(sent)
               #print(s.__dict__)
                f = s.test_root()
                if f is not None:
                  self.facts.append(f)
              #print(facts)
                        #print(text)
    return 0

  def export_csv(self, name='loh.csv'):
    f_out = open(name, 'w')
    for fact in self.facts:
      f_out.write(fact.to_text()+'\n')
    f_out.close()
    return 0

  def export_rdf(self, name='loh.csv'):
    return 0

  def export_tuples(self):
    tuples = list()
    for fact in self.facts:
      tuples.append(fact.to_tuple())
    return tuples

In [42]:
class Ontology:
  def __init__(self, facts):
    d_obj = dict()
    d_rel = dict()
    for t in facts:
      if t[0] in d_obj:
        d_obj[t[0]] = d_obj[t[0]] + 1
      else: 
        d_obj[t[0]] = 1
      if t[2] in d_obj:
        d_obj[t[2]] = d_obj[t[2]] + 1
      else: 
        d_obj[t[2]] = 1
      if t[1] in d_rel:
        d_rel[t[1]] = d_rel[t[1]] + 1
      else: 
        d_rel[t[1]] = 1
    self.correct_obj = [obj for obj in d_obj if d_obj[obj] > 1]
    self.correct_rel = [rel for rel in d_rel if d_rel[rel] > 1]
    self.triplets = set([fact for fact in facts if fact[0] in self.correct_obj and fact[2] in self.correct_obj])
    #correct_facts = [fact for fact in l if fact[0] in correct_obj or fact[2] in correct_obj]
    #correct_facts = [fact for fact in l if fact[1] in correct_rel]
  
  def export_RDF(self):
    f = open('test.rdf', 'w')
    f.write(header)
    for triplet in self.triplets:
      f.write(rel_temp.format('_'.join(triplet[1].split()), '_'.join(triplet[0].split()), '_'.join(triplet[2].split())))
    for obj in self.correct_obj:
      f.write(obj_temp.format('_'.join(obj.split())))
    f.write(end)
    f.close()
    return

In [36]:
header = \
'''<?xml version="1.0"?>
<rdf:RDF xmlns="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont/"
     xml:base="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont/"
     xmlns:owl="http://www.w3.org/2002/07/owl#"
     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
     xmlns:xml="http://www.w3.org/XML/1998/namespace"
     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"
     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
    <owl:Ontology rdf:about="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont">
        <rdfs:comment>Тестовая попытка</rdfs:comment>
    </owl:Ontology>  
'''
end = "</rdf:RDF>"

obj_temp = \
'''
<owl:Class rdf:about="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont#{}"/>
'''

rel_temp = \
'''
    <owl:ObjectProperty rdf:about="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont#{}">
        <rdfs:domain rdf:resource="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont#{}"/>
        <rdfs:range rdf:resource="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont#{}"/>
    </owl:ObjectProperty>
'''

In [24]:
 w = 'дли на ху я'

In [26]:
'_'.join(w.split())

'дли_на_ху_я'

In [ ]:
test = FactExtractor()
test.extract()
#test.export_csv('страховая.csv')
l = test.export_tuples()
onto = Ontology(l)
onto.export_RDF()

In [23]:
onto.export_RDF()

In [ ]:
#f = open("1.txt")
#text = f.read()
facts = list()
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
#sent = doc.sents[0]
for sent in doc.sents:
  s = Sentence(sent)
  #print(s.__dict__)
  f = s.test_root()
  if f is not None:
    facts.append(f)
  print(facts)

In [ ]:
for fact in facts:
  if len(fact.to_text()) < 100:
    print(fact.to_text())

In [5]:
header

'\n<?xml version="1.0"?>\n<rdf:RDF xmlns="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont/"\n     xml:base="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont/"\n     xmlns:owl="http://www.w3.org/2002/07/owl#"\n     xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\n     xmlns:xml="http://www.w3.org/XML/1998/namespace"\n     xmlns:xsd="http://www.w3.org/2001/XMLSchema#"\n     xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">\n    <owl:Ontology rdf:about="http://www.semanticweb.org/georgxxl/ontologies/2023/3/testont">\n        <rdfs:comment>Тестовая попытка</rdfs:comment>\n    </owl:Ontology>  \n'

In [3]:
cringe.format("жорикс")

'ты лох, жорикс'

In [ ]:
#text = "Машинное обучение (англ. machine learning, ML) — это класс методов искусственного интеллекта, характерной чертой которых является не прямое решение задачи, а обучение за счёт применения решений множества сходных задач. Для построения таких методов используются средства математической статистики, численных методов, математического анализа, методов оптимизации, теории вероятностей, теории графов, различные техники работы с данными в цифровой форме."
#text = "Машинное обучение используется для решения задач, которые помогают бизнесу и упрощают жизнь"
#text = "Язык используется для кунилингуса, который является лаской, которая используется для получения наслаждения."
#text = "Лодейников собирается придумывать план"
#text = "Лодейников считает, что нужно придумать план"
#text = "Лодейников думает, что язык - часть тела человека"
#text = "Российская противотанковая мина нажимного действия"
#text = "Котя и Пёся - спортсмены и отличники"
#text = "Бил, Гей, используется для поцелуев"
#text = "Язык - часть тела, которая используются для поцелуев"
#text = "Когда петухи поют, в городе наступает ночь"
#text = "Непервый пирог оказался вкусным, когда мы его съели"
#text = "Я не буду делать ни эту математику, ни этот русский"
#text = "Ах, как красиво тут!"
text = "Противотанковая мина нажимного действия используется для уничтожения техники противника."

In [ ]:
#f = open("1.txt")
#text = f.read()
doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)
sent = doc.sents[1]
print(sent)

DocSent(start=192, stop=413, text='Было\tсделано\tнемало\tнаучных\tоткрытий,\tинжене..., tokens=[...])


In [ ]:
sent.syntax.print()

                  ┌► Было           aux:pass
┌───────────────┌─└─ сделано        
│             ┌─└──► немало         nsubj:pass
│             │   ┌► научных        amod
│   ┌─┌─────┌─└──►└─ открытий       obl
│   │ │     │   ┌──► ,              punct
│   │ │     │   │ ┌► инженерных     amod
│   │ │     └──►└─└─ изобретений    conj
│   │ │       ┌────► ,              punct
│   │ │       │   ┌► на             case
│ ┌►│ │       │ ┌─└─ основе         obl
│ │ │ │       │ └──► которых        nmod
│ │ │ │       └─┌─── создавались    acl:relcl
│ │ │ │       │ │ ┌► целые          amod
│ │ │ │       │ └►└─ отрасли        nsubj:pass
│ │ │ │ ┌─────└─└──► промышленности nmod
│ │ │ │ │     │   ┌► (              punct
│ │ │ │ │ ┌─┌─└──►└─ машиностроение parataxis
│ │ │ │ │ │ │     ┌► ,              punct
│ │ │ │ │ │ └────►└─ энергетика     conj
│ │ │ │ │ │     ┌──► ,              punct
│ │ │ │ │ │     │ ┌► цифровые       amod
│ │ │ └►│ │     └─└─ технологии     conj
│ │ │   │ │       ┌► и          

In [ ]:
sent.syntax.tokens

[SyntaxToken(
     id='2_1',
     text='Было',
     head_id='2_2',
     rel='aux:pass'
 ),
 SyntaxToken(
     id='2_2',
     text='сделано',
     head_id='2_0',
     rel='root'
 ),
 SyntaxToken(
     id='2_3',
     text='немало',
     head_id='2_2',
     rel='nsubj:pass'
 ),
 SyntaxToken(
     id='2_4',
     text='научных',
     head_id='2_5',
     rel='amod'
 ),
 SyntaxToken(
     id='2_5',
     text='открытий',
     head_id='2_3',
     rel='obl'
 ),
 SyntaxToken(
     id='2_6',
     text=',',
     head_id='2_8',
     rel='punct'
 ),
 SyntaxToken(
     id='2_7',
     text='инженерных',
     head_id='2_8',
     rel='amod'
 ),
 SyntaxToken(
     id='2_8',
     text='изобретений',
     head_id='2_5',
     rel='conj'
 ),
 SyntaxToken(
     id='2_9',
     text=',',
     head_id='2_13',
     rel='punct'
 ),
 SyntaxToken(
     id='2_10',
     text='на',
     head_id='2_11',
     rel='case'
 ),
 SyntaxToken(
     id='2_11',
     text='основе',
     head_id='2_33',
     rel='obl'
 ),
 SyntaxTo

--------------------------------------------------------------ПОМОЙКА------------------------------------------------------------

In [ ]:
#Попытка извлечение бич-фактов 1
import os

def FactExtract(file_path: str = "") -> list:
  terms = list()
  np_chunking = ['NOUN', 'ADJ', 'ADP']
  for dirpath, dirnames, filenames in os.walk(os.path.join(".", file_path)):
    for filename in filenames:
        #print("Файл:", os.path.join(dirpath, filename))
        with open(os.path.join(dirpath, filename), 'r') as f:
          text = f.read()
          doc = nlp(text)
          cur_term = list()
          cur_poses = set()
          is_cur_term = False
          obj1 = ""
          relation = ""
          for token in doc:
            if str(token.pos_) == "VERB":
              relation = token.text
            elif str(token.pos_) in np_chunking:
              cur_term.append(token.text)
              cur_poses.add(token.pos_)
              is_cur_term = True
              print(token.text, token.lemma_, token.pos_, token.is_stop)
            
            else:
              if is_cur_term:
                print('<-')
                is_cur_term = False
                if "NOUN" in cur_poses:
                  if obj1 and relation:
                    terms.append((obj1, relation, ' '.join(cur_term)))
                    obj1 = ""
                  else:
                    obj1 = ' '.join(cur_term)
                    relation = ""
                  #terms.append(' '.join(cur_term))
                cur_term = list()
                cur_poses = set()
            #print(token.text, token.lemma_, token.pos_, token.is_stop)
            #print(token.pos_)
          if "NOUN" in cur_poses:
              terms.append(' '.join(cur_term))
  return terms

facts = FactExtract("Folder1")

In [ ]:
facts

In [ ]:
for dirpath, dirnames, filenames in os.walk("./Folder1"):
    # перебрать каталоги
    #for dirname in dirnames:
    #    print("Каталог:", os.path.join(dirpath, dirname))
    # перебрать файлы
    for filename in filenames:
        print("Файл:", os.path.join(dirpath, filename))
        with open(os.path.join(dirpath, filename), 'r') as f:
          text = f.read()
          print(text)

In [ ]:
os.listdir(os.path.join(".", 'Folder1'))

['.ipynb_checkpoints', '2.txt', '3.txt', '1.txt']

In [ ]:
for t1, t2, t3 in os.walk("."):
  print(t1, t2, t3)

In [ ]:
f = open("./Folder1/1.txt")
print(f.read())

In [ ]:
#Тестируем модуль построения OWL
test_relations1 = [
    ("Мина нажимного действия", "Является", "Мина"),
    ("Война", "Это", "Конфликт"),
    ("Оружие", "Используется для", "Уничтожение живой силы")
]

test_relations2 = [
    ("Мина нажимного действия", "Is", "Мина"),
    ("Война", "Is", "Конфликт"),
    ("Оружие", "IsUsedFor", "Уничтожение живой силы")
]

construct_OWL(test_relations1, "test1.txt")
construct_OWL(test_relations2, "test2.txt")

In [ ]:
#Попытка извлечение терминов 1
import os

def TermExtract(file_path: str = "") -> list:
  terms = list()
  np_chunking = ['NOUN', 'ADJ', 'ADP']
  for dirpath, dirnames, filenames in os.walk(os.path.join(".", file_path)):
    for filename in filenames:
        #print("Файл:", os.path.join(dirpath, filename))
        with open(os.path.join(dirpath, filename), 'r') as f:
          text = f.read()
          doc = nlp(text)
          cur_term = list()
          cur_poses = set()
          is_cur_term = False
          for token in doc:
            if str(token.pos_) in np_chunking:
              cur_term.append(token.text)
              cur_poses.add(token.pos_)
              is_cur_term = True
              print(token.text, token.lemma_, token.pos_, token.is_stop)
            else:
              if is_cur_term:
                print('<-')
                is_cur_term = False
                if "NOUN" in cur_poses:
                  terms.append(' '.join(cur_term))
                cur_term = list()
                cur_poses = set()
            #print(token.text, token.lemma_, token.pos_, token.is_stop)
            #print(token.pos_)
          if "NOUN" in cur_poses:
              terms.append(' '.join(cur_term))
  return terms

terms = TermExtract("Folder1")

In [ ]:
def construct_OWL(relations: list, file_name: str = "output.txt"):
  f = open(file_name, 'w')
  for relation in relations:
    f.write(str(relation)+'\n')
  f.close()
  return